In [59]:
import argparse
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from scipy import sparse
import os
import pandas as pd
from scipy import sparse
import numpy as np
from typing import Tuple

In [60]:
SEED = 777
VERBOSE = False

In [61]:
torch.manual_seed(SEED)

#만약 GPU가 사용가능한 환경이라면 GPU를 사용
if torch.cuda.is_available():
    use_cuda = True

device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [95]:
print("Load Movielens dataset")
# Load Data
DATA_DIR = '/opt/ml/input/data/train'
raw_data = pd.read_csv(os.path.join(DATA_DIR, 'train_ratings.csv'), header=0)

Load Movielens dataset


In [96]:
_min_cnt = float('inf')
for usr_id, tp in raw_data.groupby('user', as_index=False):
    usr_id, tp['item']
    if not isinstance(usr_id, int) and not isinstance(tp, pd.DataFrame):
        print('except')
        break
    _min_cnt = min(tp.shape[0], _min_cnt)

In [97]:
_min_cnt

16

In [98]:
raw_data.shape[0]

5154471

In [135]:
def generate_general_train_test_set(test_plays: pd.DataFrame, n_all=10, n_seq=2) -> Tuple[pd.DataFrame, pd.DataFrame]:
    np.random.seed(SEED)
    trains, labels = [], []
    for usr_id, tp in test_plays.groupby('user', as_index=False):
        _n_all = min(tp.shape[0]//4, n_all)
        _n_seq = min(_n_all, n_seq)
        _n_static = _n_all - _n_seq
        _n_all = _n_static + _n_seq

        _idxs = np.random.permutation(tp.shape[0]-_n_seq)[:_n_static]
        _mask = tp.index.isin(tp.index[_idxs])
        for i in range(_n_seq):
            _mask[-i-1] = True
        if VERBOSE:
         if _n_all != 10:
            print('_n_all:', _n_all)
            print(usr_id, _idxs)
            print(_n_static, _n_seq)

        trains.append(tp[~_mask])
        labels.append(tp[_mask])
        
    train_df = pd.concat(trains)
    label_df = pd.concat(labels)
    return train_df, label_df

In [136]:
train_df, label_df = generate_general_train_test_set(raw_data, n_all=10, n_seq=2)

In [132]:
assert raw_data.shape[0] == train_df.shape[0] + label_df.shape[0]

In [134]:
label_df.head(12)

,user,item,time
61,11,150,1230785343
89,11,60037,1230787639
131,11,57368,1230788571
134,11,1748,1230788594
257,11,6996,1230856754
275,11,364,1230858919
301,11,3000,1230859482
351,11,69526,1251170492
374,11,7153,1294796132
375,11,4226,1294796159


In [138]:
label_df['user'].unique()

array([    11,     14,     18, ..., 138486, 138492, 138493])

In [141]:
label_df[label_df['user'] == 11]['item']

61       150
89     60037
131    57368
134     1748
257     6996
275      364
301     3000
351    69526
374     7153
375     4226
Name: item, dtype: int64

In [143]:
test_submission = pd.read_csv("general_test_submission.csv")

In [145]:
test_submission[test_submission['user'] == 11]['item']

0     1680
1     1610
2     2683
3     4370
4    80463
5    88125
6     4741
7     1252
8      915
9     1237
Name: item, dtype: int64

In [151]:
label_df[label_df['user'] == 11]['item'].isin(test_submission[test_submission['user'] == 11]['item']).sum()

0

In [157]:
from tqdm import tqdm

In [158]:
_cnt = 0
for user in tqdm(label_df['user'].unique()):
    _cnt += label_df[label_df['user'] == user]['item'].isin(test_submission[test_submission['user'] == user]['item']).sum()

100%|██████████| 31360/31360 [00:42<00:00, 741.46it/s]


In [154]:
_cnt

28304

In [161]:
_cnt/test_submission.shape[0]

0.09025510204081633